In [1]:
%reset -f

In [18]:
from pathlib import Path
from sssom.parsers import parse_sssom_json, parse_sssom_table
from sssom.writers import write_table, write_json


In [6]:
ofn_file = Path("__file__").resolve().parents[1] /"omim-ordo-do" / "boomer_output" / "mondo_exact.ofn"
ofn_json = Path("__file__").resolve().parents[1] /"omim-ordo-do" / "boomer_output" / "mondo_exact.json"
! robot convert -i {ofn_file} --format json -o {ofn_json}

In [14]:
input_msdf = parse_sssom_table(Path("__file__").resolve().parents[1] /"omim-ordo-do" / "boomer_input" / "mondo_exact" / "combined.sssom.tsv")
input_msdf.df.head()


In [15]:
input_msdf.df.head()

,subject_id,subject_label,predicate_id,object_id,mapping_justification,confidence
0,MONDO:0000001,disease or disorder,skos:exactMatch,SCTID:64572001,semapv:UnspecifiedMatching,0.99
1,MONDO:0000001,disease or disorder,skos:exactMatch,Orphanet:377788,semapv:UnspecifiedMatching,0.99
2,MONDO:0000001,disease or disorder,skos:exactMatch,NCIT:C2991,semapv:UnspecifiedMatching,0.99
3,MONDO:0000001,disease or disorder,skos:exactMatch,DOID:4,semapv:UnspecifiedMatching,0.99
4,MONDO:0000001,disease or disorder,skos:exactMatch,UMLS:C0012634,semapv:UnspecifiedMatching,0.99


In [24]:
! sssom parse {ofn_json} -I obographs-json -o test.tsv

sssom.util.NoCURIEException: http://identifiers.org/meddra/C1535926 does not follow any known prefixes
